In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [5]:
import tensorflow as tf
import numpy as np
# from path_explain.path_explainer_tf import PathExplainerTF

In [6]:
x = tf.constant([[2.0, 5.0], [6.0, 8.0]])

In [7]:
with tf.GradientTape() as t:
    t.watch(x)
    y = tf.square(x)
    z = tf.reduce_sum(y, axis=-1)
t.gradient(z, x)

<tf.Tensor: id=17, shape=(2, 2), dtype=float32, numpy=
array([[ 4., 10.],
       [12., 16.]], dtype=float32)>

In [8]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1, input_shape=(2,)))

In [9]:
model.weights

[<tf.Variable 'dense/kernel:0' shape=(2, 1) dtype=float32, numpy=
 array([[-0.5324452 ],
        [-0.42080152]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [10]:
k = tf.constant([[200.0, 500.0], [6.0, 8.0]])

In [11]:
with tf.GradientTape() as t:
    t.watch(k)
    y = model(k)
t.gradient(y, k)

<tf.Tensor: id=63, shape=(2, 2), dtype=float32, numpy=
array([[-0.5324452 , -0.42080152],
       [-0.5324452 , -0.42080152]], dtype=float32)>

In [48]:
x = tf.constant([[2.0, 5.0, 3.0], [6.0, 8.0, 1.0]])

In [60]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(10, input_shape=(3,), activation='softmax'))
model.add(tf.keras.layers.Dense(1))

In [98]:
with tf.GradientTape() as second_order_tape:
    second_order_tape.watch(x)
    with tf.GradientTape() as first_order_tape:
        first_order_tape.watch(x)
        z = model(x)
    grad = first_order_tape.gradient(z, x)
jacobian = second_order_tape.jacobian(grad, x)
# second_grad = second_order_tape.gradient(indexed_grad, x)

In [104]:
jacobian.numpy()[np.arange(2), :, np.arange(2), :]

array([[[-0.02110989,  0.01286735,  0.01510115],
        [ 0.01286735, -0.0107625 , -0.00567085],
        [ 0.01510115, -0.00567085, -0.01363467]],

       [[ 0.02816404, -0.01690738, -0.01374249],
        [-0.01690738,  0.01197595,  0.00869031],
        [-0.01374249,  0.00869031,  0.0086968 ]]], dtype=float32)

In [71]:
second_grad

<tf.Tensor: id=2047, shape=(2, 3), dtype=float32, numpy=
array([[ 0.01286735, -0.0107625 , -0.00567085],
       [-0.01690738,  0.01197595,  0.00869031]], dtype=float32)>

In [50]:
grad

<tf.Tensor: id=1093, shape=(2, 3), dtype=float32, numpy=
array([[ 4., 10.,  6.],
       [12., 16.,  2.]], dtype=float32)>

In [37]:
x

<tf.Tensor: id=0, shape=(2, 2), dtype=float32, numpy=
array([[2., 5.],
       [6., 8.]], dtype=float32)>

In [38]:
z

<tf.Tensor: id=360, shape=(2,), dtype=float32, numpy=array([ 29., 100.], dtype=float32)>

In [39]:
grad

<tf.Tensor: id=374, shape=(2, 2), dtype=float32, numpy=
array([[ 4., 10.],
       [12., 16.]], dtype=float32)>

In [40]:
tape.gradient(grad, x)

In [13]:
num_samples = 10
explainer = PathExplainerTF(model)
eg_single = explainer._single_attribution(x[0], np.zeros((1, 2)).astype(np.float32),
                            current_alphas=np.linspace(0.0, 1.0, num_samples).astype(np.float32),
                            num_samples=num_samples, batch_size=50,
                            use_expectation=False, output_index=0, input_indices=None)

NameError: name 'PathExplainerTF' is not defined

In [12]:
eg_single

array([0.5297718, 5.702946 ], dtype=float32)

In [13]:
model.weights[0][:, 0] * x[0]

<tf.Tensor: id=117, shape=(2,), dtype=float32, numpy=array([0.5297718, 5.7029467], dtype=float32)>

In [14]:
model(x[0:1])

<tf.Tensor: id=125, shape=(1, 1), dtype=float32, numpy=array([[6.2327185]], dtype=float32)>

In [15]:
eg_single

array([0.5297718, 5.702946 ], dtype=float32)

In [105]:
x

<tf.Tensor: id=1076, shape=(2, 3), dtype=float32, numpy=
array([[2., 5., 3.],
       [6., 8., 1.]], dtype=float32)>